In [ ]:
import os
import numpy as np

In [ ]:
a = np.load("cache/beit_embeddings.npz")
beit_embeddings = a["arr_0"]
beit_shape = beit_embeddings.shape

In [ ]:
full_embedding = beit_embeddings
beit_embeddings
print(full_embedding.shape)

In [ ]:
small_array = full_embedding[::10]
print("small_array.shape = ", small_array.shape)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

pca = PCA(n_components=8)
scaler = StandardScaler()
x = scaler.fit_transform(small_array)

pca.fit(x)

In [ ]:
import matplotlib.pyplot as plt
exp_var_pca = pca.explained_variance_ratio_
#
# Cumulative sum of eigenvalues; This will be used to create step plot
# for visualizing the variance explained by each principal component.
#
cum_sum_eigenvalues = np.cumsum(exp_var_pca)
#
# Create the visualization plot
#
plt.bar(range(0,len(exp_var_pca)), exp_var_pca, alpha=0.5, align='center', label='Individual explained variance')
plt.step(range(0,len(cum_sum_eigenvalues)), cum_sum_eigenvalues, where='mid',label='Cumulative explained variance')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal component index')
plt.legend(loc='best')
plt.show()
print(cum_sum_eigenvalues)

In [ ]:
xx = scaler.transform(full_embedding)
del full_embedding
pca_embedding_1 = pca.transform(xx[:len(xx)//3])
print(1)
pca_embedding_2 = pca.transform(xx[len(xx)//3:2*(len(xx)//3)])
print(2)
pca_embedding_3 = pca.transform(xx[2*(len(xx)//3):])
print(3)
pca_embedding = np.concatenate((pca_embedding_1, pca_embedding_2, pca_embedding_3), dtype=np.float32)

In [ ]:
print(pca_embedding[1795:1805, :5])

In [ ]:
mean_embedding = np.zeros_like(pca_embedding)
for i in range(len(pca_embedding) // 1800):
    mean_embedding[i*1800: (i+1)*1800] = np.mean(pca_embedding[i*1800: (i+1)*1800], 0)
print(mean_embedding[1795:1805, :5])

In [ ]:
full_pca_embedding = np.load("cache/pca_embedding_beit_simclr_handcrafted_weighted_by_average_motion_and_feature,simclr=0.5_handcrafted=1.8.npy")

In [ ]:
final_submission = np.concatenate((full_pca_embedding[:,:120], mean_embedding[:,:]), axis=1)
print(final_submission[1795:1805, :5])
final_submission.shape

In [ ]:
DATA_DIR = "/data/behavior-representation/"
def validate_submission(submission):
    frame_number_map = np.load(os.path.join(DATA_DIR, 'frame_number_map.npy'), allow_pickle=True).item()

    if not isinstance(submission, np.ndarray):
        print("Embeddings should be a numpy array")
        return False
    elif not len(submission.shape) == 2:
        print("Embeddings should be 2D array")
        return False
    elif not submission.shape[1] <= 128:
        print("Embeddings too large, max allowed is 128")
        return False
    elif not isinstance(submission[0, 0], np.float32):
        print(f"Embeddings are not float32")
        return False

    
    total_clip_length = frame_number_map[list(frame_number_map.keys())[-1]][1]
            
    if not len(submission) == total_clip_length:
        print(f"Emebddings length doesn't match submission clips total length")
        return False

    if not np.isfinite(submission).all():
        print(f"Emebddings contains NaN or infinity")
        return False

    print("All checks passed")
    return True

In [ ]:
validate_submission(final_submission)
np.save(f'cache/final_submission.npy', final_submission)